In [ ]:
#pip install yfinance
#pip install pandas
#pip install matplotlib
#pip install tqdm

In [ ]:
# 取得する銘柄コードを格納
Scraping_Stocks = [
    "8411",
]

In [ ]:
# data_listに株価取得
import yfinance
import pandas as pd
from tqdm import tqdm

data_list = []
for code in tqdm(Scraping_Stocks):
    tmp = yfinance.download(f"{str(code)}.T", progress=False)
    tmp["code"] = code
    data_list.append(tmp)

In [ ]:
df = pd.concat(data_list) #データフレームの結合
df #データの確認

In [ ]:
# "Adj Close" = 調整後終値
import matplotlib.pyplot as plt
df["Adj Close"].plot()
plt.show()
"""調整後終値 = 株式分割前の終値を株式分割後の終値に調整したもの"""

In [ ]:
# 25日移動平均線を基準とする
window = 25  # 移動平均のウィンドウサイズ
df['MA'] = df['Adj Close'].rolling(window=window).mean() #移動平均線
df['StdDev'] = df['Adj Close'].rolling(window=25).std() #標準偏差
df['Deviation'] = (df['Adj Close'] - df['MA']) / df['StdDev'] #移動平均線からの乖離率
df['Bollinger bands+2'] = df['MA'] + (2*df['StdDev']) #ボリンジャーバンド+2σ
df['Bollinger bands-2'] = df['MA'] - (2*df['StdDev']) #ボリンジャーバンド-2σ

In [ ]:
df # データを視覚的に確認する

In [ ]:
# プロットの範囲指定
start_date = '2022-01-01'
end_date = '2023-06-06'

# 範囲指定のためのスライシング
filtered_data = df.loc[start_date:end_date]

In [ ]:
# 標準偏差データのプロット
filtered_data['StdDev'].plot()
plt.show() # プロットの表示

In [ ]:
# 調整後終値、移動平均線、ボリンジャーバンドデータのプロット
filtered_data[['Bollinger bands+2','MA','Bollinger bands-2','Adj Close']].plot()
plt.show() # プロットの表示

In [ ]:
# 移動平均線からの乖離率データのプロット
filtered_data[['Deviation']].plot()
plt.show() # プロットの表示

In [ ]:
# 今日をTimestampオブジェクトに変換
import datetime
today = datetime.date.today()
today = pd.Timestamp(today)  # 日付をTimestampオブジェクトに変換

In [ ]:
# 売買シグナル、ポジション変数を初期化
df['Signal'] = 0
df['Position'] = 0

In [ ]:
# バックテストの実行
for i in range(25, len(df)): #24行目以前は 移動平均線等が NaN のため25行目から最終行まで
    if df.index[i] == pd.Timestamp(today.date()):
        break

    df.loc[df.index[i], 'Position'] = df.loc[df.index[i-1], 'Position']
    if df['Adj Close'].iloc[i] <= df['Bollinger bands-2'].iloc[i]:
        if df['Position'].iloc[i] == 0:  # ポジションがない場合にのみ買いシグナルを発生
            df.loc[df.index[i], 'Position'] = 1
            df.loc[df.index[i], 'Signal'] = 1
        elif df['Position'].iloc[i] == -1:  # 売りポジションがある場合には売りポジションをクローズ
            df.loc[df.index[i], 'Position'] = 0
            df.loc[df.index[i], 'Signal'] = 1
    elif df['Adj Close'].iloc[i] >= df['Bollinger bands+2'].iloc[i]:
        if df['Position'].iloc[i] == 0:  # ポジションがある場合にのみ売りシグナルを発生
            df.loc[df.index[i], 'Position'] = -1  # ポジションを売りに切り替え
            df.loc[df.index[i], 'Signal'] = -1
        elif df['Position'].iloc[i] == 1:  # 買いポジションがある場合には買いポジションをクローズ
            df.loc[df.index[i], 'Position'] = 0
            df.loc[df.index[i], 'Signal'] = -1

df = df.drop(df.index[i+1:])  # i行目以降の行を削除

In [ ]:
#標準偏差 +2以上 or -2以下
filtered_data = df[(df['Deviation'] >= 2) | (df['Deviation'] <= -2)][['Deviation','Signal','Position']]
filtered_data.tail(10)

In [ ]:
"""リターンの算出"""
# 初期資本 100万年、1ポジション = 100株
capital = 1000000  # 初期資本
position_size = 100  # 1ポジションの株数

In [ ]:
"""株数変化列、株式資産、キャッシュ残高、総資産、リターン、累積リターンの算出"""
#df['Position'] = df['Position'].shift()  # シグナルを1日シフトしてポジション列を作成
#df['Position'].fillna(0, inplace=True)  # ポジション列の最初の値を0に設定

df['Shares'] = df['Position'] - df['Position'].shift()  # 株数変化列を作成
df['Shares'].fillna(0, inplace=True)  # 株数変化列の最初の値を0に設定

df['Portfolio'] = df['Adj Close'] * df['Position'] * position_size  # ポートフォリオ価値列を作成
df['Portfolio'].iloc[0] = 0  # 最初の日のポートフォリオ価値を0に設定

df['Cash'] = capital - (df['Shares'] * df['Adj Close'] * position_size).cumsum()  # キャッシュ残高列を作成
df['Value'] = df['Cash'] + df['Portfolio']  # 総資産列を作成

df['Returns'] = df['Value'].pct_change()  # リターン列を作成
df['Cumulative Returns'] = (1 + df['Returns']).cumprod()  # 累積リターン列を作成

"""
Shares = 株数変化列
Portfolio = 株式資産
Cash = キャッシュ残高
Value = 総資産
Returns = リターン
Cumulative Returns = 累積リターン
"""

In [ ]:
# バックテスト結果の表示
print('Cumulative Returns:', df['Returns'].sum())
print('Return:', df['Returns'].sum() * capital)  # リターンの計算

In [ ]:
# プロットの設定
plt.plot(df[['Returns','Cumulative Returns']])
plt.legend(df[['Returns','Cumulative Returns']])
plt.show()

In [ ]:
# desktopにdf.csvとして保存
import os
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
df.to_csv(os.path.join(desktop_path, 'df.csv'), index=True)